# Assignment 3

Import libraries and define common helper functions

In [59]:
import os
import sys
import gzip
import json
from pathlib import Path
import csv

import pandas as pd
import s3fs
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
import fastavro
import pygeohash
import snappy
import jsonschema
from jsonschema.exceptions import ValidationError
from jsonschema import validate


endpoint_url='https://storage.budsc.midwest-datascience.com'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)


def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'routes.jsonl.gz'
    with open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        

    return records

Load the records from https://storage.budsc.midwest-datascience.com/data/processed/openflights/routes.jsonl.gz 

In [60]:
records = read_jsonl_data()

## 3.1

### 3.1.a JSON Schema

In [61]:
def validate_jsonl_data(records):
    schema_path = schema_dir.joinpath('routes-schema.json')
    validation_csv_path = results_dir.joinpath('validation-results.csv')
    with open(schema_path) as f:
        schema = json.load(f)
        
    with open(validation_csv_path, 'w') as f:    
        for i, record in enumerate(records):
            try:
                # Validate record against the schema
                validate(record, schema)
                f.write(f"Record {i+1},Valid\n") 
                pass
            except ValidationError as e:
                ## Print message if invalid record
                f.write(f"Record {i+1},Invalid: {str(e)}\n")
                pass
            

validate_jsonl_data(records)

### 3.1.b Avro

In [62]:
def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')
    with open(schema_path) as f:
        schema = fastavro.parse_schema(json.load(f))

    with open(data_path, 'wb') as f:
        fastavro.writer(f, schema, records)
            
create_avro_dataset(records)

### 3.1.c Parquet

In [63]:
def create_parquet_dataset():
    src_data_path = 'routes.jsonl.gz'
    parquet_output_path = results_dir.joinpath('routes.parquet')
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    
    with open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            pass
            ## TODO: Use Apache Arrow to create Parquet table and save the dataset
            # Load JSON lines data
            records = [json.loads(line) for line in f]

            # Convert records to PyArrow table
            table = pa.Table.from_pandas(pd.DataFrame(records))

            # Write table to Parquet file
            pq.write_table(table, parquet_output_path)

create_parquet_dataset()

### 3.1.d Protocol Buffers

In [64]:
sys.path.insert(0, os.path.abspath('routes_pb2'))

import routes_pb2

def _airport_to_proto_obj(airport):
    obj = routes_pb2.Airport()
    if airport is None:
        return None
    if airport.get('airport_id') is None:
        return None

    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj


def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    if not airline.get('name'):
        return None
    if not airline.get('airline_id'):
        return None
    obj.airline_id = airline.get('airline_id')
    obj.name = airline.get('name')
    if airline.get('alias'):
        obj.alias = airline.get('alias')
    # Verify if 'active' attribute exists
    if 'active' in airline:
        obj.active = airline['active']
    return obj

def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()
        airline = _airline_to_proto_obj(record.get('airline', {}))
        if airline:
            route.airline.CopyFrom(airline)
        src_airport = _airport_to_proto_obj(record.get('src_airport', {}))
        if src_airport:
            route.src_airport.CopyFrom(src_airport)
        ## TODO
        dst_airport = _airport_to_proto_obj(record.get('dst_airport', {}))
        if dst_airport:
            route.dst_airport.CopyFrom(dst_airport)
        route.codeshare = record['codeshare']
        equipment = record.get('equipment')
        routes.route.append(route)
    data_path = results_dir.joinpath('routes.pb')
    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    compressed_path = results_dir.joinpath('routes.pb.snappy')
    
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
        
create_protobuf_dataset(records)

In [67]:
routes = routes_pb2.Routes()
# Define the directory path where results will be stored
results_dir = 'results'

# Create the directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

# Define the data paths
data_path = os.path.join(results_dir, 'routes.pb')
compressed_path = os.path.join(results_dir, 'routes.pb.snappy')

# Serialize and write the data
with open(data_path, 'wb') as f:
    f.write(routes.SerializeToString())

with open(compressed_path, 'wb') as f:
    f.write(snappy.compress(routes.SerializeToString()))

# Measure file sizes
pb_size = os.path.getsize(data_path)
compressed_size = os.path.getsize(compressed_path)

def write_comparison_to_csv(csv_path, pb_size, compressed_size):
    with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Format', 'Size (bytes)'])
        writer.writerow(['Protocol Buffers', pb_size])
        writer.writerow(['Protocol Buffers (Compressed)', compressed_size])

# Write results to CSV
csv_path = os.path.join(results_dir, 'comparison.csv')
write_comparison_to_csv(csv_path, pb_size, compressed_size)

## 3.2

### 3.2.a Simple Geohash Index

In [ ]:
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    ## TODO: Create hash index
            
create_hash_dirs(records)

### 3.2.b Simple Search Feature

In [ ]:
def airport_search(latitude, longitude):
    ## TODO: Create simple search to return nearest airport
    pass
    
airport_search(41.1499988, -95.91779)